In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
kdtree = None
radius = 1.3

def init_kdtree(tree):
    global kdtree
    kdtree = tree

def find_plane_directions(indices, points, radius=2):
    normals = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        neighbors = points[neighbor_indices]
        mean = np.mean(neighbors, axis=0)
        norm = neighbors - mean
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eigh(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_vec = eig_vec[:, sorted_idx]
        plane_direction = eig_vec[:, 2]

        # # Flip normal if needed
        # reference_normal = plane_direction  # Use its own normal as a reference
        # if np.dot(reference_normal, np.array([0, 0, 1])) < 0:  # Ensuring upward direction
        #     plane_direction = -plane_direction

        normals.append(plane_direction)

    return normals  # Return aligned normals

def calculate_normal_variation(indices, points, normals, radius=2):
    eigenvalues = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        neighbor_normals = normals[neighbor_indices]
        mu = np.mean(neighbor_normals, axis=0)
        norm = neighbor_normals - mu
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eigh(cov)
        sorted_idx = np.argsort(eig_val)[::-1]

        # Compute dominant normal (first PCA component)
        dominant_normal = eig_vec[:, sorted_idx[2]]  # Last column

        # Align normals
        aligned_normals = np.array(neighbor_normals)
        for i in range(len(aligned_normals)):
            if np.dot(dominant_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        # Compute covariance after alignment
        cov_after = np.cov(aligned_normals.T)
        eig_val_after, _ = np.linalg.eigh(cov_after)
        sorted_idx_after = np.argsort(eig_val_after)[::-1]
        if np.linalg.cond(cov) > 10000:  # Experiment with threshold
            eig_val_after[1] = 0  # Force λ₂ to 0 in unstable cases
        else:
            eig_val_after = eig_val_after[sorted_idx_after] / np.sum(eig_val_after)

        # Store eigenvalues
        eigenvalues.append(eig_val_after)

    return eigenvalues  # Return eigenvalues instead of storing in shared array

if __name__ == "__main__": 
    dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
    pcd = o3d.io.read_point_cloud(dataname)
    pcd = util.preProcessCloud(pcd)
    myarray = np.asarray(pcd.points)
    indices = np.arange(len(myarray))
    kdtree = cKDTree(myarray)
    num_chunks = 16
    chunk_size = len(myarray) // num_chunks
    chunked_indices = [indices[i:i + chunk_size] for i in range(0, len(indices), chunk_size)]

    # # Single-core execution check
    # results_single = find_plane_directions(chunked_indices[0], myarray)
    # print("First 5 normals (single-core):")
    # print(results_single[:5])

    # First pass: Compute normals
    start_time_first_pca = time.time()
    with multiprocessing.Pool(processes=4, initializer=init_kdtree, initargs=(kdtree,)) as pool:
        normals_chunks = pool.starmap(find_plane_directions, [(chunk_indices, myarray, radius) for chunk_indices in chunked_indices])
    first_pca_duration = time.time() - start_time_first_pca
    print(f"First PCA time: {first_pca_duration:.2f} seconds")

    # Flatten normals to a single array in order
    all_normals = np.vstack(normals_chunks)

    # # Single-core execution check
    # results_single = calculate_normal_variation(chunked_indices[0], myarray, all_normals)
    # print("First 5 eigenvalues (single-core):")
    # print(results_single[:5])

    # Second pass: Compute normal variation
    start_time_second_pca = time.time()
    with multiprocessing.Pool(processes=4, initializer=init_kdtree, initargs=(kdtree,)) as pool:
        eigenvalues_chunks = pool.starmap(calculate_normal_variation, [(chunk_indices, myarray, all_normals, radius) for chunk_indices in chunked_indices])
    second_pca_duration = time.time() - start_time_second_pca
    print(f"Second PCA time: {second_pca_duration:.2f} seconds")

    # Flatten eigenvalues to a single array in order
    all_eigenvalues = np.vstack(eigenvalues_chunks)
    lambda2_values = all_eigenvalues[:, 1]

    # Print a few results to verify
    print("First 5 normals:")
    print(all_normals[:5])
    print("First 5 eigenvalues:")
    print(all_eigenvalues[:5])

First PCA time: 119.31 seconds
Second PCA time: 474.04 seconds
First 5 normals:
[[ 1.63706886e-02 -4.28153068e-03  9.99856824e-01]
 [ 9.81775780e-03 -4.16497540e-03  9.99943131e-01]
 [ 5.19251839e-03 -5.49079091e-03  9.99971444e-01]
 [ 1.97441588e-03 -3.88094137e-03  9.99990520e-01]
 [ 3.98232665e-04 -1.84545855e-03  9.99998218e-01]]
First 5 eigenvalues:
[[4.72983683e-06 0.00000000e+00 7.31529154e-01]
 [4.29813128e-06 0.00000000e+00 8.82054901e-01]
 [5.08639641e-06 0.00000000e+00 9.91580136e-01]
 [5.60202311e-06 0.00000000e+00 9.37663679e-01]
 [6.42319544e-06 0.00000000e+00 1.00472745e+00]]


In [3]:
class GaussMapVisualizer:
    def __init__(self, pcd, kdtree, all_normals, lambda2_values, radius):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)
        self.pcd.paint_uniform_color([0.6, 0.6, 0.6])
        self.kdtree = kdtree
        self.plane_directions = all_normals
        self.radius = radius
        self.lambda2_values = lambda2_values
        low, high = np.percentile(self.lambda2_values, [95, 100])
        self.core_indices = np.where((self.lambda2_values > low) & (self.lambda2_values <= high))[0]
        print(f"Found {len(self.core_indices)} core points out of {len(self.pcd.points)} total points.")
        self.reference_normal = None

        self.current_index = 0
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window("GaussMapVisualizer")

        self.vis.register_key_callback(262, self.next_neighborhood)
        self.vis.register_key_callback(264, self.show_random_core_point)
        self.vis.add_geometry(self.pcd)
        self.apply_lambda2_heatmap()
        self._update_neighborhood()

    def get_nearest_neighbor_directions(self, point, kdtree, pcd, plane_directions, radius=2):
        """ Get the directions of the k nearest neighbors to a given point. """
        idx = kdtree.query_ball_point(point, radius)
        nearest_points = np.asarray(pcd.points)[idx]
        nearest_directions = np.asarray(plane_directions)[idx]
        return idx, nearest_points, nearest_directions
    
    def create_normal_lines(self, neighbor_points, neighbor_directions, scale=0.2):
        """ Create line segments for the normal directions at each point. """
        line_set = o3d.geometry.LineSet()

        start_points = np.array(neighbor_points)
        end_points = start_points + scale * np.array(neighbor_directions)
        lines = [[start_points[i], end_points[i]] for i in range(len(neighbor_points))]
        line_set.points = o3d.utility.Vector3dVector(np.concatenate(lines, axis=0))
        line_indices = [[i, i + 1] for i in range(0, len(lines) * 2, 2)]
        line_set.lines = o3d.utility.Vector2iVector(line_indices)
        line_set.colors = o3d.utility.Vector3dVector(np.tile((0, 0, 1), (len(lines), 1)))
        return line_set
    
    def align_normals(self, reference_normal, neighbor_directions):
        aligned_normals = np.array(neighbor_directions)
        
        # Check dot product: If negative, flip the normal
        for i in range(len(aligned_normals)):
            if np.dot(reference_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        return aligned_normals
    
    def calculate_normal_variation(self, normals):
        mu = np.mean(normals, axis=0)
        norm = normals - mu
        cov = np.cov(norm.T)
        eig_val, _ = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_val_norm = eig_val / np.sum(eig_val)
        
        return mu, eig_val_norm, cov
    
    def update_gauss_map(self, normals):
        """ Update the Gauss Map visualization with the current neighborhood's normals. """
        normals = np.array(normals)
        normals /= np.linalg.norm(normals, axis=1, keepdims=True)  # Normalize to unit sphere

        # Create figure
        plt.figure("Gauss Map", figsize=(6, 6))
        plt.clf()  # Clear previous plot
        ax = plt.subplot(111, projection="3d")

        # Plot unit sphere
        u = np.linspace(0, 2 * np.pi, 30)
        v = np.linspace(0, np.pi, 20)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color="gray", alpha=0.3, edgecolor="none")  # Transparent sphere

        # Plot normal vectors
        for normal in normals:
            ax.quiver(0, 0, 0, normal[0], normal[1], normal[2], color="b", linewidth=1, arrow_length_ratio=0.1)

        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.set_title(f"Gauss Map - Neighborhood {self.current_index}")

        plt.pause(0.1)  # Allow Matplotlib to update

    def show_random_core_point(self, vis):
        if len(self.core_indices) == 0:
            print("No core points found.")
            return

        # Pick a random core point
        self.current_index = random.choice(self.core_indices)

        # Visualize its neighborhood
        self._update_neighborhood()

        print(f"Showing core point {self.current_index}.")

    def _update_neighborhood(self):
        """ Update visualization for the current neighborhood. """
        # Get the currently selected point
        query_point = np.asarray(self.pcd.points)[self.current_index]

        # Get nearest neighbors
        idx, neighbor_points, neighbor_directions = self.get_nearest_neighbor_directions(query_point, self.kdtree, self.pcd, self.plane_directions, radius=self.radius)
        if self.current_index==0:
            self.reference_normal = neighbor_directions[0]

        #aligned_directions = self.align_normals(self.reference_normal, neighbor_directions)
        normal_mean, normal_variation, cov_after = self.calculate_normal_variation(neighbor_directions)

        if hasattr(self, "normal_lines"):
            self.vis.remove_geometry(self.normal_lines)
        self.normal_lines = self.create_normal_lines(neighbor_points, neighbor_directions, scale=2)
        self.vis.add_geometry(self.normal_lines)
        view_ctl = self.vis.get_view_control()
        lookat = query_point
        zoom = 0.080000000000000002
        front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
        up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
        view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
        view_ctl.set_up(up)      # Set the up direction of the camera
        view_ctl.set_front(front)  # Set the front direction of the camera
        view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

        #self.update_gauss_map(aligned_directions)

        self.vis.update_geometry(self.pcd)
        print(f"Neighborhood {self.current_index}/{len(self.pcd.points)} updated", flush=True)
        print(f'Aligned normals: {neighbor_directions}', flush=True)
        print(f"Normal mean: {normal_mean}, Normal variation: {normal_variation}", flush=True)
        print(f"Std Dev of Normals: {np.std(neighbor_directions, axis=0)}")
        print(f"Condition Number of Covariance: {np.linalg.cond(cov_after)}")
        print(15*"-", flush=True)

    def next_neighborhood(self, vis):
        """ Move to the next neighborhood when right arrow key is pressed. """
        self.current_index = (self.current_index + 500) % len(self.pcd.points)
        self._update_neighborhood()
    
    def normalize_lambda2_colors(self, lambda2_values):
        """ Normalize λ2 values to a colormap range. """
        min_val, max_val = np.percentile(lambda2_values, [2, 98])
        norm_lambda2 = (lambda2_values - min_val) / (max_val - min_val + 1e-6)  # Normalize to [0,1]

        # Use a colormap (e.g., viridis)
        colors = cm.viridis(norm_lambda2)[:, :3]  # Extract RGB colors

        return colors

    def apply_lambda2_heatmap(self):
        colors = self.normalize_lambda2_colors(self.lambda2_values)

        self.pcd.colors = o3d.utility.Vector3dVector(colors)  # Assign colors to points

        #self.vis.update_geometry(self.pcd)
        print("Heatmap applied!")

    def run(self):
        self.vis.run()  # Start the visualization loop (blocks until closed)
        self.vis.destroy_window()

In [4]:
viewer = GaussMapVisualizer(pcd, kdtree, all_normals, lambda2_values, radius)
viewer.run()

Found 105714 core points out of 2114266 total points.
Heatmap applied!
Neighborhood 0/2114266 updated
Aligned normals: [[ 1.86377227e-02 -9.17016335e-03  9.99784248e-01]
 [ 1.21959177e-02 -2.75765203e-03  9.99921824e-01]
 [ 9.98306063e-03 -4.00840543e-03  9.99942134e-01]
 [-1.55815180e-02  2.81483989e-03 -9.99874639e-01]
 [ 1.44578535e-02 -3.99192803e-04  9.99895400e-01]
 [ 1.52096815e-02  8.59662997e-04  9.99883957e-01]
 [ 1.30036433e-02  2.36595892e-03  9.99912650e-01]
 [ 1.57276195e-02  1.79985100e-03  9.99874693e-01]
 [ 1.88221226e-02 -1.13498375e-03  9.99822204e-01]
 [ 2.69942403e-02 -1.63841237e-03  9.99634246e-01]
 [ 2.11156274e-02 -1.97166530e-03  9.99775096e-01]
 [ 2.28560565e-02 -1.60846232e-03  9.99737472e-01]
 [ 2.16999067e-02  4.68630378e-03  9.99753546e-01]
 [ 1.69558023e-02  3.02494878e-03  9.99851664e-01]
 [ 1.90737048e-02 -5.67896135e-03  9.99801952e-01]
 [ 2.21603533e-02  4.95268546e-03  9.99742162e-01]
 [ 2.63289429e-02 -2.74830375e-03  9.99649555e-01]
 [ 2.00378473e